In [72]:
library(dplyr)
library(tidyr)
library(tibble)
library(qvalue)
library(data.table)

source("/mnt/lareaulab/reliscu/code/fisher_test.R")

setwd("/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/tasic_2018/mouse_ALM")

Here I perform enrichment analysis to find modules enriched for cell type markers. 

These modules will later be used to correlate with exon PSI to define cell type-specific exons.

In [73]:
get_module_enrichments <- function(network_dir, ctype_genes, mod_def="PosFDR") {
    # Traverse networks to get cell type enrichments for each module
    networks <- list.dirs(file.path(getwd(), network_dir), full.names=TRUE, recursive=FALSE)
    networks <- networks[lengths(lapply(networks, list.files)) > 0]

    enrichments_list <- lapply(seq_along(networks), function(i) {
        kME_path <- list.files(networks[i])[grep("kME", list.files(networks[i]))]
        kME <- fread(file.path(networks[i], kME_path), data.table=FALSE)
        mod_col <- grep("PosFDR", colnames(kME))
        mod_genes <- tapply(kME$Gene, kME[,mod_col], list)
        
        if (length(mod_genes) > 0) {
            all_genes <- kME$Gene

            # For each module: calculate enrichment for DE genes from each cell type
            mod_enrichments_list <- lapply(mod_genes, function(mod) {
                lapply(unlist(lapply(ctype_genes, function(set) {
                    fisher_test(set, mod, all=all_genes)
                })), c)
            })
            
            # Save the network the module came from
            network_id <- sapply(strsplit(networks[i], "/"), function(x) x[length(x)])
            mod_enrichments_df <- reshape2::melt(mod_enrichments_list)
            colnames(mod_enrichments_df) <- c("Pval", "Cell_type", "Module")
            
            # Save path to module eigengenes table for downstream analyses
            ME_path <- list.files(networks[i])[grep("eigengene", list.files(networks[i]))]
            
            data.frame(
                Network=network_id,
                kME_path=file.path(networks[i], kME_path),
                ME_path=file.path(networks[i], ME_path),
                mod_enrichments_df
            )
        }
    })
    enrichments_df <- do.call(rbind, enrichments_list)
    enrichments_df$Qval <- qvalue(enrichments_df$Pval)$qvalue

    enrichments_df
}

In [74]:
network_dir <- "tasic_2018_ALM_STAR_20pcntCells_0.4pcntVar_200samples_log2_pseudobulk_Modules" # "tasic_2018_ALM_STAR_20pcntCells_0.4pcntVar_200samples_log2_pseudobulk_Modules"
network_dataset <- "20pcntCells_0.4pcntVar_200samples_log2_pseudobulk"

In [ ]:
mod_def <- "PosFDR"
unique <- TRUE

### Get DE genes frome each cell type: 1 vs. pooled tests

In [76]:
pooled_res_list <- readRDS("data/tasic_2018_ALM_STAR_donor_cell_type_pseudobulk_1_vs_pooled_DE_genes_dream.RDS")

In [77]:
pval_threshold <- .05/length(pooled_res_list)
lfc_threshold <- 6

pooled_ctype_genes <- lapply(pooled_res_list, function(df) {
    mask <- (df['adj.P.Val'] < pval_threshold) & (abs(df['logFC']) > lfc_threshold)
    df[mask, 1]
})
names(pooled_ctype_genes) <- names(pooled_res_list) 

In [78]:
if (unique) {
    all_genes <- unlist(pooled_ctype_genes)
    duplicates <- unique(names(table(all_genes)[table(all_genes) > 1]))

    # Remove markers that are not unique to a cell type

    pooled_ctype_genes <- lapply(pooled_ctype_genes, function(x) x[!(x %in% duplicates)])
}

In [79]:
data.frame(No.genes=lengths(pooled_ctype_genes)) %>% arrange(-`No.genes`)

,No.genes
,<int>
Peri,1568
Macrophage,395
Endo,304
Astro,282
VLMC,136
SMC,129
Oligo,105
Meis2,83
CR,61


In [ ]:
pooled_enrichments_df <- get_module_enrichments(network_dir, pooled_ctype_genes, mod_def)

In [ ]:
# Get most enriched cell type for each module
# If cell type is most enriched in multiple modules, choose module with smallest p-value

pooled_top_mods_df <- pooled_enrichments_df %>%
    group_by(Network, Module) %>%
    slice_min(Qval) %>%
    group_by(Cell_type) %>%
    arrange(Network) %>%
    slice_min(Qval, with_ties=FALSE) %>%
    filter(Qval < .05) %>%
    arrange(Qval)

In [ ]:
pooled_top_mods_df[,c("Cell_type", "Pval", "Qval", "Module", "Network")]

Cell_type,Pval,Qval,Module,Network
<chr>,<dbl>,<dbl>,<chr>,<chr>
Macrophage,8.582124e-186,5.641373e-181,black,Bicor-None_signum0.928_minSize4_merge_ME_0.9_20151
Endo,1.284248e-183,1.688376e-179,pink,Bicor-None_signum0.842_minSize4_merge_ME_0.9_20151
Astro,3.831374e-173,1.144780e-169,yellow,Bicor-None_signum0.629_minSize4_merge_ME_0.9_20151
Oligo,3.015653e-146,4.045529e-143,darkturquoise,Bicor-None_signum0.928_minSize3_merge_ME_0.9_20151
SMC,3.985193e-111,3.402112e-108,yellow,Bicor-None_signum0.928_minSize5_merge_ME_0.9_20151
VLMC,5.670062e-98,4.235407e-95,royalblue,Bicor-None_signum0.629_minSize6_merge_ME_0.9_20151
Vip,6.421378e-39,2.972556e-36,green,Bicor-None_signum0.53_minSize4_merge_ME_0.9_20151
NP,9.420120e-38,4.330225e-35,cyan,Bicor-None_signum0.842_minSize3_merge_ME_0.9_20151
Lamp5,6.081477e-31,2.324185e-28,pink,Bicor-None_signum0.53_minSize4_merge_ME_0.9_20151


### Get DE genes from each cell type: pairwise tests

In [ ]:
pairwise_res_list <- readRDS("data/tasic_2018_ALM_STAR_donor_cell_type_pseudobulk_pairwise_DE_genes_dream.RDS")

In [ ]:
ctypes <- unique(sapply(strsplit(names(pairwise_res_list), "_vs_"), "[", 1))

pairwise_ctype_genes <- lapply(ctypes, function(target) {
    # Subset to pairwise tests with target cell type
    ctype_res_list <- pairwise_res_list[grep(paste0("^", target), names(pairwise_res_list))]

    # For each pairwise test, return genes that meet p-value threshold:
    pval_threshold <- .05 /length(ctype_res_list)
    ctype_genes_list <- lapply(ctype_res_list, function(df) {
        mask <- df$adj.P.Val < pval_threshold
        df[mask, 1]
    }) 

    # Restrict to genes that were identified in EVERY pairwise test
    Reduce(intersect, ctype_genes_list)
})
names(pairwise_ctype_genes) <- ctypes

In [ ]:
data.frame(No.genes=lengths(pairwise_ctype_genes)) %>% arrange(-`No.genes`)

,No.genes
,<int>
Macrophage,333
Astro,288
Endo,237
Oligo,131
VLMC,85
SMC,60
Pvalb,31
Lamp5,22
CR,21


In [ ]:
pairwise_ctype_genes <- pairwise_ctype_genes[lengths(pairwise_ctype_genes) > 0]

In [ ]:
if (unique) {
    all_genes <- unlist(pairwise_ctype_genes)
    duplicates <- unique(names(table(all_genes)[table(all_genes) > 1]))

    # Remove markers that are not unique to a cell type

    pairwise_ctype_genes <- lapply(pairwise_ctype_genes, function(x) x[!(x %in% duplicates)])
}

pairwise_ctype_genes <- pairwise_ctype_genes[lengths(pairwise_ctype_genes) > 0]

In [ ]:
pairwise_enrichments_df <- get_module_enrichments(network_dir, pairwise_ctype_genes, mod_def)

In [ ]:
# Get most enriched cell type for each module
# If cell type is most enriched in multiple modules, choose module with smallest p-value

pairwise_top_mods_df <- pairwise_enrichments_df %>%
    group_by(Network, Module) %>%
    slice_min(Qval) %>%
    group_by(Cell_type) %>%
    arrange(Network) %>%
    slice_min(Qval, with_ties=FALSE) %>%
    filter(Qval < .05) %>%
    arrange(Qval)

In [ ]:
pairwise_top_mods_df[,c("Cell_type", "Pval", "Qval", "Module", "Network")]

Cell_type,Pval,Qval,Module,Network
<chr>,<dbl>,<dbl>,<chr>,<chr>
Endo,0.000000e+00,0.000000e+00,pink,Bicor-None_signum0.465_minSize10_merge_ME_0.9_20151
Macrophage,0.000000e+00,0.000000e+00,magenta,Bicor-None_signum0.53_minSize10_merge_ME_0.9_20151
Astro,3.644304e-299,9.165570e-296,yellow,Bicor-None_signum0.842_minSize3_merge_ME_0.9_20151
Oligo,2.198702e-188,1.868184e-185,darkturquoise,Bicor-None_signum0.928_minSize3_merge_ME_0.9_20151
VLMC,2.207268e-144,1.779284e-141,orange,Bicor-None_signum0.629_minSize8_merge_ME_0.9_20151
SMC,7.478725e-87,4.394695e-84,royalblue,Bicor-None_signum0.629_minSize10_merge_ME_0.9_20151
Pvalb,8.037187e-38,3.158414e-35,brown,Bicor-None_signum0.53_minSize8_merge_ME_0.9_20151
Sst,6.370102e-31,2.176775e-28,green,Bicor-None_signum0.928_minSize3_merge_ME_0.9_20151
Lamp5,5.682781e-30,1.890532e-27,red,Bicor-None_signum0.842_minSize5_merge_ME_0.9_20151


In [ ]:
# pairwise_top_mods_df[,c("Cell_type", "Pval", "Qval", "Module", "Network")]

### Get DE genes frome each cell type: 1 vs. mean analysis

In [ ]:
mean_res_list <- readRDS("data/tasic_2018_ALM_STAR_donor_cell_type_pseudobulk_1_vs_meanOthers_DE_genes_dream.RDS")

In [ ]:
pval_threshold <- .05/length(mean_res_list)
lfc_threshold <- 6

mean_ctype_genes <- lapply(mean_res_list, function(df) {
    mask <- (df['adj.P.Val'] < pval_threshold) & (abs(df['logFC']) > lfc_threshold)
    df[mask, 1]
})
names(mean_ctype_genes) <- names(mean_res_list) 

In [ ]:
if (unique) {
    all_genes <- unlist(mean_ctype_genes)
    duplicates <- unique(names(table(all_genes)[table(all_genes) > 1]))

    # Remove markers that are not unique to a cell type

    mean_ctype_genes <- lapply(mean_ctype_genes, function(x) x[!(x %in% duplicates)])
}

In [ ]:
data.frame(No.genes=lengths(mean_ctype_genes)) %>% arrange(-`No.genes`)

,No.genes
,<int>
Peri,1806
Endo,879
Macrophage,828
VLMC,551
SMC,542
Astro,503
Oligo,228
Vip,67
Sncg,66


In [ ]:
mean_enrichments_df <- get_module_enrichments(network_dir, mean_ctype_genes, mod_def)

In [ ]:
# Get most enriched cell type for each module
# If cell type is most enriched in multiple modules, choose module with smallest p-value

mean_top_mods_df <- mean_enrichments_df %>%
    group_by(Network, Module) %>%
    slice_min(Qval) %>%
    group_by(Cell_type) %>%
    arrange(Network) %>%
    slice_min(Qval, with_ties=FALSE) %>%
    filter(Qval < .05) %>%
    arrange(Qval)

In [ ]:
mean_top_mods_df[,c("Cell_type", "Pval", "Qval", "Module", "Network")]

Cell_type,Pval,Qval,Module,Network
<chr>,<dbl>,<dbl>,<chr>,<chr>
Macrophage,2.937780e-244,1.931120e-239,green,Bicor-None_signum0.928_minSize5_merge_ME_0.9_20151
Endo,6.215654e-215,4.085798e-211,pink,Bicor-None_signum0.842_minSize4_merge_ME_0.9_20151
Astro,4.910968e-208,1.793431e-204,yellow,Bicor-None_signum0.465_minSize5_merge_ME_0.9_20151
Oligo,2.543546e-175,2.933289e-172,lightgreen,Bicor-None_signum0.842_minSize6_merge_ME_0.9_20151
SMC,1.814868e-131,1.472821e-128,yellow,Bicor-None_signum0.928_minSize5_merge_ME_0.9_20151
VLMC,4.718012e-111,3.264567e-108,orange,Bicor-None_signum0.629_minSize8_merge_ME_0.9_20151
NP,4.158145e-41,1.898136e-38,cyan,Bicor-None_signum0.842_minSize3_merge_ME_0.9_20151
Vip,6.406931e-39,2.845630e-36,green,Bicor-None_signum0.53_minSize4_merge_ME_0.9_20151
L5_IT,2.095423e-38,9.182703e-36,turquoise,Bicor-None_signum0.53_minSize4_merge_ME_0.9_20151


### Compare enrichment results

In [ ]:
pooled_top_mods_df$Network_short <- gsub("Bicor-None_", "", gsub("_merge_ME_0.9_20151", "", pooled_top_mods_df$Network))
pairwise_top_mods_df$Network_short <- gsub("Bicor-None_", "", gsub("_merge_ME_0.9_20151", "", pairwise_top_mods_df$Network))
mean_top_mods_df$Network_short <- gsub("Bicor-None_", "", gsub("_merge_ME_0.9_20151", "", mean_top_mods_df$Network))

cols <- c("Cell_type", "Qval", "Module", "Network_short")

all_top_mods_df <- merge(
    merge(
        pooled_top_mods_df[,cols], 
        pairwise_top_mods_df[,cols], 
        by="Cell_type", all=TRUE
    ), 
    mean_top_mods_df[,cols], by="Cell_type", all=TRUE
)

In [ ]:
all_top_mods_df <- all_top_mods_df[,order(colnames(all_top_mods_df))]
all_top_mods_df

Cell_type,Module,Module.x,Module.y,Network_short,Network_short.x,Network_short.y,Qval,Qval.x,Qval.y
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
Astro,yellow,yellow,yellow,signum0.465_minSize5,signum0.629_minSize4,signum0.842_minSize3,1.793431e-204,1.144780e-169,9.165570e-296
Endo,pink,pink,pink,signum0.842_minSize4,signum0.842_minSize4,signum0.465_minSize10,4.085798e-211,1.688376e-179,0.000000e+00
L2_3_IT,orangered3,orangered3,mediumpurple2,signum0.53_minSize10,signum0.53_minSize10,signum0.465_minSize5,4.751775e-09,4.835030e-13,9.758999e-06
L5_IT,turquoise,turquoise,turquoise,signum0.53_minSize4,signum0.629_minSize10,signum0.629_minSize4,9.182703e-36,6.917617e-13,3.433212e-12
L5_PT,paleturquoise,salmon4,salmon4,signum0.842_minSize4,signum0.629_minSize4,signum0.629_minSize4,1.417690e-11,5.328785e-10,8.204811e-03
L6_CT,thistle2,thistle2,thistle2,signum0.53_minSize10,signum0.53_minSize10,signum0.53_minSize10,3.326398e-28,3.175082e-24,4.034197e-06
L6_IT,orange,darkorange2,darkorange2,signum0.629_minSize10,signum0.842_minSize3,signum0.842_minSize3,1.181262e-20,3.840624e-08,1.250952e-15
L6b,purple2,purple2,NA,signum0.465_minSize5,signum0.465_minSize5,NA,5.081986e-07,4.643165e-07,NA
Lamp5,red,pink,red,signum0.465_minSize5,signum0.53_minSize4,signum0.842_minSize5,4.169363e-30,2.324185e-28,1.890532e-27


### Select the module from the lowest Qval per cell type

In [ ]:
# Select the module from the lowest Qval per cell type

pooled_top_mods_df$DE_Test <- "Pooled"
pairwise_top_mods_df$DE_Test <- "Pairwise"
mean_top_mods_df$DE_Test <- "Mean"

all_top_mods_df <- rbind(pooled_top_mods_df, pairwise_top_mods_df, mean_top_mods_df)

In [ ]:
colnames(all_top_mods_df)

[1] "Network"       "kME_path"      "ME_path"       "Pval"         
[5] "Cell_type"     "Module"        "Qval"          "Network_short"
[9] "DE_Test"

In [ ]:
top_qval_mods_df <- all_top_mods_df %>%
    group_by(Cell_type) %>%
    slice_min(Qval)

top_qval_mods_df[, c(cols, "DE_Test")]

Cell_type,Qval,Module,Network_short,DE_Test
<chr>,<dbl>,<chr>,<chr>,<chr>
Astro,9.165570e-296,yellow,signum0.842_minSize3,Pairwise
Endo,0.000000e+00,pink,signum0.465_minSize10,Pairwise
L2_3_IT,4.835030e-13,orangered3,signum0.53_minSize10,Pooled
L5_IT,9.182703e-36,turquoise,signum0.53_minSize4,Mean
L5_PT,1.417690e-11,paleturquoise,signum0.842_minSize4,Mean
L6_CT,3.326398e-28,thistle2,signum0.53_minSize10,Mean
L6_IT,1.181262e-20,orange,signum0.629_minSize10,Mean
L6b,4.643165e-07,purple2,signum0.465_minSize5,Pooled
Lamp5,4.169363e-30,red,signum0.465_minSize5,Mean


In [ ]:
if (unique) {
    write.csv(top_qval_mods_df, file=paste0("data/tasic_2018_ALM_STAR_donor_cell_type_pseudobulk_DE_genes_dream_unique_", network_dataset, "_", mod_def, "_top_Qval_modules.csv"), row.names=FALSE, quote=FALSE)
} else {
    write.csv(top_qval_mods_df, file=paste0("data/tasic_2018_ALM_STAR_donor_cell_type_pseudobulk_DE_genes_dream_", network_dataset, "_", mod_def, "_top_Qval_modules.csv"), row.names=FALSE, quote=FALSE)
}
